In [78]:
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import PDFPageAggregator
from io import StringIO
import os
import codecs
import re 
import tabula

import numpy as np
import camelot


In [149]:
def readPDFs():
    for country in os.listdir('Approved Plans'):
        # Test only on Cook Islands
        if country != 'Cook Islands':
            continue
        for file in os.listdir('Approved Plans/'+country):

            if file.startswith('MDR'):
                output_string = StringIO()
                with open('Approved Plans/'+country+'/'+file, 'rb') as fh:
                    parser = PDFParser(fh)
                    doc = PDFDocument(parser)
                    rsrcmgr = PDFResourceManager()
                    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    landscapepages = list()
                    for ix, page in enumerate(PDFPage.create_pages(doc)):
                        # only pages with landscape orientation
                        if page.cropbox[2] < 800:
                            continue
                        interpreter.process_page(page)
                        # count landscape page 
                        landscapepages.append(ix+1)
                    
                    landscapepages = str(landscapepages)
                    landscapepages = landscapepages[1:-1]
                    # find all tables on the landscape pages and create excel tables
                    foundtables = camelot.read_pdf(file, pages=landscapepages)
                    for i in range(len(foundtables)):
                        foundtables[i].to_excel('Table_test'+str(i)+'.xlsx') 
                        
                    file1 = codecs.open('Approved Plans/'+country+'/'+country+'.txt', "w", "utf-8")  # write mode
                    file1.write(output_string.getvalue())
                    file1.close()
            if file.endswith('txt'):
                extractInfo(file,country)

In [150]:
def extractInfo(file, country):
    sectors = ['Livelihoods and basic needs', 'Water, sanitation and hygiene', 'Health']
    file1 = codecs.open('Approved Plans/' + country + '/' + file.split('.')[0] + '.txt', "r",
                        "utf-8")  # write mode
    text = file1.read()
    # get detailed operational plan
    text = re.split(r"(.*)Detailed Operational Plan", text, re.MULTILINE | re.DOTALL)[-1]

    for sector in sectors:
        text_sector = ' '.join(re.split(sector, text, re.MULTILINE | re.DOTALL))
        #print(text_sector)
        for line in re.split(r"\n", text_sector, re.MULTILINE | re.DOTALL)[:10]:
            if 'People targeted:' in line:
                peop_target = re.findall(r"[0-9,]+", line)[0]
            if 'Requirements (CHF):' in line:
                budget = re.findall(r"[0-9,]+", line)[0]
        # Here we should perhaps open the right excel pages and get out the right activities
        activity_paragraphs = re.split("Activities planned", text_sector, re.MULTILINE | re.DOTALL)
        
        #print(activity_paragraphs[1].replace('\n\n', ''))
        #print(activity_paragraphs[1].replace('\n\n',''))
       # for activity_paragraph in activity_paragraphs[1:2]:
         #   print(activity_paragraph)

In [151]:
readPDFs()